In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
import argparse
from sklearn.metrics import roc_curve as ROC
from sklearn.metrics import auc,brier_score_loss

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import dataframe_image as dfi

from Utils.loading import *
from Utils.treino import *
from Utils.init_redes import *
from Utils.KD import *
from Utils.ensemble import *
from Utils.redes import *
from Utils.data_set import *
from Utils.cobertura import *
from Utils.graficos import *

cuda


In [2]:
if torch.cuda.is_available():
    device = 'cuda'  
else:
    device = 'cpu'
print(device)

cuda


# Carregando o data set

In [3]:
data_set = "Cifar100"

if data_set == "Cifar10":
    transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader, classes = cifar10()
elif data_set == "Cifar100":
    transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader= cifar100()

Files already downloaded and verified
Files already downloaded and verified


# Cerregando as redes

In [4]:
n_rede = 3
models = load_resnet18_v2(n_rede, data_set)

# VENDO A MELHOR TEMPERATURA PARA CADA MODELO

In [5]:
best_acc_list = list()
best_rc_list = list()
best_auc_list = list()
best_auroc_list = list()
best_T_list = list()

base_acc_list = list()
base_rc_list = list()
base_auc_list = list()
base_auroc_list = list()

for i in range(1,16):
    print("Rede numero {}".format(i))
    model = load_resnet18_v2(i, data_set)    
    outputs = torch.tensor([]).to("cuda")
    labels_cat = torch.tensor([])

    correct_list = list()
    covarege_list = list()
    auc_list = list()
    auroc_list = list()

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            outputs_0 = model(images.to("cuda"))
            outputs = torch.cat((outputs,outputs_0),dim=0)
            labels_cat = torch.cat((labels_cat,labels),dim=0) # Junta as respostas

    valores = np.arange(0.01, 2.001, 0.01)

    for valor in valores:
        T = valor

        outputs1 = outputs/T
        outputs_softmax = torch.nn.functional.softmax(outputs1, dim=-1)

        uncs, _, _, predict = calculate_predicted(outputs_softmax)
        correct_list.append((1-calculate_correct(predict, labels_cat))*100)
        aurc_aux, rc_aux = AURC(predict, labels_cat.to("cuda"), uncs, return_curve=True)
        covarege_list.append(rc_aux)
        auc_list.append(aurc_aux)
        auroc_list.append(AUROC(predict, labels_cat.to("cuda"), -uncs))

    best_auc_index = np.argmin(auc_list)
    best_acc_list.append(correct_list[best_auc_index])
    best_rc_list.append(covarege_list[best_auc_index])
    best_auc_list.append(auc_list[best_auc_index])
    best_auroc_list.append(auroc_list[best_auc_index])
    best_T_list.append(valores[best_auc_index])
    
    
    base_acc_list.append(correct_list[99])
    base_rc_list.append(covarege_list[99])
    base_auc_list.append(auc_list[99])
    base_auroc_list.append(auroc_list[99])

Rede numero 1
Rede numero 2
Rede numero 3
Rede numero 4
Rede numero 5
Rede numero 6
Rede numero 7
Rede numero 8
Rede numero 9
Rede numero 10
Rede numero 11
Rede numero 12
Rede numero 13
Rede numero 14
Rede numero 15


In [16]:
for i in range(15):
    print("Rede numero {}".format(i+1))
    print("Melhor T:   {:.2f}".format(best_T_list[i]))
    print("Acuracia normal: {:.2f} || Acuracia melhor T: {:.2f}".format(base_acc_list[i],best_acc_list[i]))
    print("AUC normal:   {:.5f}  || AUC melhor T:      {:.5f}".format(base_auc_list[i],best_auc_list[i]))
    print("AUROC normal: {:.5f}  || AUROC melhor T:    {:.5f}".format(base_auroc_list[i],best_auroc_list[i]))
    print()

Rede numero 1
Melhor T:   0.67
Acuracia normal: 76.03 || Acuracia melhor T: 76.03
AUC normal:   0.06646  || AUC melhor T:      0.06592
AUROC normal: 0.87364  || AUROC melhor T:    0.87446

Rede numero 2
Melhor T:   0.61
Acuracia normal: 75.82 || Acuracia melhor T: 75.82
AUC normal:   0.06657  || AUC melhor T:      0.06633
AUROC normal: 0.87631  || AUROC melhor T:    0.87535

Rede numero 3
Melhor T:   0.85
Acuracia normal: 76.03 || Acuracia melhor T: 76.03
AUC normal:   0.06785  || AUC melhor T:      0.06760
AUROC normal: 0.86866  || AUROC melhor T:    0.86882

Rede numero 4
Melhor T:   0.78
Acuracia normal: 76.19 || Acuracia melhor T: 76.19
AUC normal:   0.06593  || AUC melhor T:      0.06546
AUROC normal: 0.87217  || AUROC melhor T:    0.87309

Rede numero 5
Melhor T:   0.88
Acuracia normal: 75.92 || Acuracia melhor T: 75.92
AUC normal:   0.06553  || AUC melhor T:      0.06526
AUROC normal: 0.87732  || AUROC melhor T:    0.87763

Rede numero 6
Melhor T:   0.84
Acuracia normal: 75.97 |

# 2 REDES

In [37]:
n_redes = 2

models = load_cifar100(n_redes)

# ENEMBLE T IGUAIS

correct_list = list()
covarege_list = list()
auc_list = list()
auroc_list = list()

outputs = torch.tensor([]).to("cuda")
labels_cat = torch.tensor([])

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data

        outputs_0 = caculate_outputs(models[0:n_redes],images.to("cuda"))
        outputs_0 = torch.stack(tuple(outputs_0),dim=0)
        outputs = torch.cat((outputs,outputs_0),dim=1)
        labels_cat = torch.cat((labels_cat,labels),dim=0) # Junta as respostas

T1 = 0.67
T2 = 0.61

output0 = outputs[0]/T1
output1 = outputs[1]/T2

outputs_stack = torch.stack((output0,output1),dim=0)

outputs_softmax = torch.nn.functional.softmax(outputs_stack, dim=-1)
outputs_mean = torch.mean(outputs_softmax, dim=0)

uncs, _, _, predict = calculate_predicted(outputs_mean)
acc = (1-calculate_correct(predict, labels_cat))*100
aurc, rc = AURC(predict, labels_cat.to("cuda"), uncs, return_curve=True)
auroc = AUROC(predict, labels_cat.to("cuda"), -uncs)


outputs_mean2 = torch.mean(outputs_stack, dim=0)
outputs_softmax2 = torch.nn.functional.softmax(outputs_mean2, dim=-1)

uncs2, _, _, predict2 = calculate_predicted(outputs_softmax2)
acc2 = (1-calculate_correct(predict2, labels_cat))*100
aurc2, rc2 = AURC(predict2, labels_cat.to("cuda"), uncs2, return_curve=True)
auroc2 = AUROC(predict2, labels_cat.to("cuda"), -uncs2)

In [41]:
print("SOFTMEAN")
print("NORMAL                  -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(77.87, 0.05758, 0.87876))
print("MELHOR T PARA CADA REDE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(acc, aurc, auroc))
print("MELHORES T PRO ENSEMBLE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(77.91, 0.05738, 0.87836))
print()
print("MEANSOFT")
print("NORMAL FALTA FAZER      -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(78.03, 0.05659, 0.88036))
print("MELHOR T PARA CADA REDE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(acc2, aurc2, auroc2))
print("MELHORES T PRO ENSEMBLE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(77.93, 0.05596, 0.88271))

SOFTMEAN
NORMAL                  -> Acuracia: 77.87 || AUC: 0.05758 || AUROC: 0.87876
MELHOR T PARA CADA REDE -> Acuracia: 77.74 || AUC: 0.05782 || AUROC: 0.87893
MELHORES T PRO ENSEMBLE -> Acuracia: 77.91 || AUC: 0.05738 || AUROC: 0.87836

MEANSOFT
NORMAL FALTA FAZER      -> Acuracia: 78.03 || AUC: 0.05659 || AUROC: 0.88036
MELHOR T PARA CADA REDE -> Acuracia: 78.07 || AUC: 0.05613 || AUROC: 0.88057
MELHORES T PRO ENSEMBLE -> Acuracia: 77.93 || AUC: 0.05596 || AUROC: 0.88271


# 3 REDES

In [30]:
n_redes = 3

models = load_cifar100(n_redes)

# ENEMBLE T IGUAIS

correct_list = list()
covarege_list = list()
auc_list = list()
auroc_list = list()

outputs = torch.tensor([]).to("cuda")
labels_cat = torch.tensor([])

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data

        outputs_0 = caculate_outputs(models[0:n_redes],images.to("cuda"))
        outputs_0 = torch.stack(tuple(outputs_0),dim=0)
        outputs = torch.cat((outputs,outputs_0),dim=1)
        labels_cat = torch.cat((labels_cat,labels),dim=0) # Junta as respostas

T1 = 0.67
T2 = 0.61
T3 = 0.85

output0 = outputs[0]/T1
output1 = outputs[1]/T2
output2 = outputs[2]/T3

outputs_stack = torch.stack((output0,output1,output2),dim=0)

outputs_softmax = torch.nn.functional.softmax(outputs_stack, dim=-1)
outputs_mean = torch.mean(outputs_softmax, dim=0)

uncs, _, _, predict = calculate_predicted(outputs_mean)
acc = (1-calculate_correct(predict, labels_cat))*100
aurc, rc = AURC(predict, labels_cat.to("cuda"), uncs, return_curve=True)
auroc = AUROC(predict, labels_cat.to("cuda"), -uncs)


outputs_mean2 = torch.mean(outputs_stack, dim=0)
outputs_softmax2 = torch.nn.functional.softmax(outputs_mean2, dim=-1)

uncs2, _, _, predict2 = calculate_predicted(outputs_softmax2)
acc2 = (1-calculate_correct(predict2, labels_cat))*100
aurc2, rc2 = AURC(predict2, labels_cat.to("cuda"), uncs2, return_curve=True)
auroc2 = AUROC(predict2, labels_cat.to("cuda"), -uncs2)

In [40]:
print("SOFTMEAN")
print("NORMAL                  -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(78.63, 0.05433, 0.88085))
print("MELHOR T PARA CADA REDE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(acc, aurc, auroc))
print("MELHORES T PRO ENSEMBLE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(78.63, 0.05406, 0.881241))
print("T1: {} || T2: {} || T3: {}".format(0.91, 0.91, 0.95))
print()
print("MEANSOFT")
print("NORMAL                  -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(78.63, 0.05432, 0.88094))
print("MELHOR T PARA CADA REDE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(acc2, aurc2, auroc2))
print("MELHORES T PRO ENSEMBLE -> Acuracia: {:.2f} || AUC: {:.5f} || AUROC: {:.5f}".format(78.95, 0.05282, 0.88007))
print("T1: {} || T2: {} || T3: {}".format(1.4, 1.40, 0.66))

SOFTMEAN
NORMAL                  -> Acuracia: 78.63 || AUC: 0.05433 || AUROC: 0.88085
MELHOR T PARA CADA REDE -> Acuracia: 77.74 || AUC: 0.05782 || AUROC: 0.87893
MELHORES T PRO ENSEMBLE -> Acuracia: 78.63 || AUC: 0.05406 || AUROC: 0.88124
T1: 0.91 || T2: 0.91 || T3: 0.95

MEANSOFT
NORMAL                  -> Acuracia: 78.63 || AUC: 0.05432 || AUROC: 0.88094
MELHOR T PARA CADA REDE -> Acuracia: 78.07 || AUC: 0.05613 || AUROC: 0.88057
MELHORES T PRO ENSEMBLE -> Acuracia: 78.95 || AUC: 0.05282 || AUROC: 0.88007
T1: 1.4 || T2: 1.4 || T3: 0.66
